
# 1. Data reading

reading the "Pigments" Dataset, provided in an emial

First load the libraries / modules.

In [0]:
# Load the needed python libraries by executing this python code (press ctrl enter)
%pip install numpy scipy matplotlib pandas
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import pandas as pd

Load the dataset into a dataframe.

In [0]:
import pandas as pd

# URL zur Datei (GitHub)
url = "https://raw.githubusercontent.com/Tao-Pi/CAS-Applied-Data-Science-GroupWork/refs/heads/main/data%2C%20calculations%2C%20results/Pigments_Version202509051820.csv"

# CSV einlesen und leere Zeilen überspringen
df = pd.read_csv(url, skip_blank_lines=True)

# Ausgabe prüfen
print(df.head())
print(f"\nShape: {df.shape}")


Browse through all rows.

In [0]:
pd.set_option('display.max_rows', 200)
df

change col names to more descriptive names

In [0]:
df.rename(columns={
    'name': 'sample',
    'RABD670': 'green pigments: index',
    'TChl-a': 'green pigments: direct concentration measurement (ug/g)',
    'locality': 'locality',
    'OC': 'organic carbon content in %'
}, inplace=True)
df

Print some descriptive statistics, using pandas summary.

In [0]:
df.describe()

# 02 Descriptive Stats

In [0]:
# === Descriptive statistics & plots for:
#     - "green pigments: index"
#     - "green pigments: direct concentration measurement (ug/g)"
# Append this block AFTER your df.rename(...) and df.describe().

from scipy import stats  # already imported as scipy.stats; this just gives 'stats' alias if needed

# Column aliases (matches your renamed columns)
col_x = "green pigments: index"
col_y = "green pigments: direct concentration measurement (ug/g)"

# Pairwise-clean data
df_pair = df.dropna(subset=[col_x, col_y]).copy()

# ---------- Descriptive statistics ----------
def describe_series(s: pd.Series) -> pd.Series:
    s_clean = s.dropna()
    out = pd.Series(dtype="float64")
    out["count"] = s_clean.shape[0]
    out["missing"] = s.shape[0] - s_clean.shape[0]
    out["missing_%"] = 100 * (1 - s_clean.shape[0] / s.shape[0]) if s.shape[0] else float("nan")
    out["mean"] = s_clean.mean()
    out["std"] = s_clean.std(ddof=1)
    out["cv (std/mean)"] = (out["std"] / out["mean"]) if out["mean"] not in (0, None) else float("nan")
    out["min"] = s_clean.min()
    out["q1"] = s_clean.quantile(0.25)
    out["median"] = s_clean.median()
    out["q3"] = s_clean.quantile(0.75)
    out["iqr"] = out["q3"] - out["q1"]
    out["max"] = s_clean.max()
    out["skewness"] = stats.skew(s_clean, bias=False)
    out["kurtosis (excess)"] = stats.kurtosis(s_clean, fisher=True, bias=False)
    if 3 <= len(s_clean) <= 5000:
        W, p = stats.shapiro(s_clean)
        out["shapiro_W"] = W
        out["shapiro_p"] = p
    else:
        out["shapiro_W"] = float("nan")
        out["shapiro_p"] = float("nan")
    return out

desc_table = pd.DataFrame({
    col_x: describe_series(df[col_x]),
    col_y: describe_series(df[col_y]),
})
print("\n=== Descriptive statistics ===")
print(desc_table.round(4))

# ---------- Correlation tests ----------
pearson_r, pearson_p = stats.pearsonr(df_pair[col_x], df_pair[col_y])
spearman_r, spearman_p = stats.spearmanr(df_pair[col_x], df_pair[col_y])
kendall_tau, kendall_p = stats.kendalltau(df_pair[col_x], df_pair[col_y])

corr_tbl = pd.DataFrame(
    {
        "statistic": ["Pearson r", "Spearman ρ", "Kendall τ"],
        "value": [pearson_r, spearman_r, kendall_tau],
        "p_value": [pearson_p, spearman_p, kendall_p],
    }
)
print("\n=== Correlation tests (pairwise complete) ===")
print(corr_tbl.round(6))




In [0]:
# === Plot 01: Histogram and box plot for the measures of pigments
#     (Index and Direct concentration measurement of green pigments (µg/g)) in different samples

# cm to inches conversion
cm_to_inch = 1/2.54
fig_width = 24.2 * cm_to_inch
fig_height = 10.08 * cm_to_inch

fig, axes = plt.subplots(2, 2, figsize=(fig_width, fig_height))
fig.suptitle(
    "Plot 01: Histogram and box plot for the measures of pigments\n"
    "(Index and Direct concentration measurement of green pigments (µg/g)) in different samples",
    fontsize=11,
    weight="bold"
)

col_x = "green pigments: index"
col_y = "green pigments: direct concentration measurement (ug/g)"

# Histogram for index
axes[0,0].hist(df[col_x].dropna(), bins=20)
axes[0,0].set_title("Histogram: Green pigments index", fontsize=9)
axes[0,0].set_xlabel(col_x, fontsize=8)
axes[0,0].set_ylabel("Frequency", fontsize=8)

# Boxplot for index
axes[1,0].boxplot(df[col_x].dropna(), vert=True)
axes[1,0].set_title("Boxplot: Green pigments index", fontsize=9)
axes[1,0].set_ylabel(col_x, fontsize=8)

# Histogram for direct concentration
axes[0,1].hist(df[col_y].dropna(), bins=20)
axes[0,1].set_title("Histogram: Direct chlorophyll concentration (µg/g)", fontsize=9)
axes[0,1].set_xlabel(col_y, fontsize=8)
axes[0,1].set_ylabel("Frequency", fontsize=8)

# Boxplot for direct concentration
axes[1,1].boxplot(df[col_y].dropna(), vert=True)
axes[1,1].set_title("Boxplot: Direct chlorophyll concentration (µg/g)", fontsize=9)
axes[1,1].set_ylabel(col_y, fontsize=8)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()


# Hypotheses
# 
**H₀ (Null Hypothesis)**: The green pigment index and the direct chlorophyll concentration are not correlated.

**H₁ (Alternative Hypothesis)**: There is a statistically significant correlation between the pigment index and the direct chlorophyll concentration.

In [0]:
import scipy.stats as stats# Drop missing values for the test

df_clean = df.dropna(subset=[
    "green pigments: index",
    "green pigments: direct concentration measurement (ug/g)"
])

# Define variables
x = df_clean["green pigments: index"]
y = df_clean["green pigments: direct concentration measurement (ug/g)"]

# Hypothesis test: Pearson correlation
corr, p_value = stats.pearsonr(x, y)

print("Pearson correlation coefficient:", corr)
print("p-value:", p_value)

# Interpret the result
alpha = 0.05
if p_value < alpha:
    print("Reject H0: There is a significant correlation.")
else:
    print("Fail to reject H0: No significant correlation.")

# Visualization
plt.scatter(x, y, alpha=0.7)
plt.xlabel("index")
plt.ylabel("direct concentration measurement (µg/g)")
plt.title("Relationship between pigment index and direct concentration measurement")

# Add regression line
slope, intercept, r_value, p_val, std_err = stats.linregress(x, y)
plt.plot(x, slope*x + intercept, color="red", label=f"Linear fit (R²={r_value**2:.2f})")
plt.legend()
plt.show()

In [0]:
import statsmodels.api as sm

# Prepare data for statsmodels
X = df_clean["green pigments: index"]
y = df_clean["green pigments: direct concentration measurement (ug/g)"]
X_with_const = sm.add_constant(X)

# Fit OLS model
model = sm.OLS(y, X_with_const).fit()

# Get influence measures
influence = model.get_influence()
leverage = influence.hat_matrix_diag
residuals = influence.resid_studentized_internal
cooks = influence.cooks_distance[0]  # Cook's distance

# Thresholds for "high" leverage (commonly 2 * (p / n))
p = X_with_const.shape[1]          # number of parameters (including intercept)
n = len(df_clean)
leverage_thresh = 2 * p / n

# Plot residuals vs. leverage
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(leverage, residuals, alpha=0.7)
ax.set_xlabel("Leverage")
ax.set_ylabel("Studentized Residuals")
ax.set_title("Residuals vs. Leverage")
ax.axhline(0, color='gray', linestyle='--', linewidth=1)

# Annotate points with high leverage (and optionally high Cook's distance)
import random

for i, (lev, res) in enumerate(zip(leverage, residuals)):
    if lev > leverage_thresh:
        row_idx = df_clean.index[i]
        jitter_x = lev + np.random.uniform(-0.01, 0.01)
        jitter_y = res + np.random.uniform(-0.1, 0.1)
        ax.text(jitter_x, jitter_y, str(row_idx), fontsize=8, ha='right', va='bottom')
for i, (lev, res) in enumerate(zip(leverage, residuals)):
    if lev > leverage_thresh:
        row_idx = df_clean.index[i]
        ax.text(lev, res, str(row_idx), fontsize=8, ha='right', va='bottom')

plt.tight_layout()
plt.show()

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import ProbPlot
plt.style.use("seaborn-v0_8") # pretty matplotlib plots

def graph(formula, x_range, label=None):
    """
    Helper function for plotting cook's distance lines
    """
    x = x_range
    y = formula(x)
    plt.plot(x, y, label=label, lw=1, ls='--', color='red')


def diagnostic_plots(X, y, model_fit=None):
  """
  Function to reproduce the 4 base plots of an OLS model in R.

  ---
  Inputs:

  X: A numpy array or pandas dataframe of the features to use in building the linear regression model

  y: A numpy array or pandas series/dataframe of the target variable of the linear regression model

  model_fit [optional]: a statsmodel.api.OLS model after regressing y on X. If not provided, will be
                        generated from X, y
  """

  model_fit = sm.OLS(y, sm.add_constant(X)).fit()

  # create dataframe from X, y for easier plot handling
  dataframe = pd.concat([X, y], axis=1)

  # model values
  model_fitted_y = model_fit.fittedvalues
  # model residuals
  model_residuals = model_fit.resid
  # normalized residuals
  model_norm_residuals = model_fit.get_influence().resid_studentized_internal
  # absolute squared normalized residuals
  model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
  # absolute residuals
  model_abs_resid = np.abs(model_residuals)
  # leverage, from statsmodels internals
  model_leverage = model_fit.get_influence().hat_matrix_diag
  # cook's distance, from statsmodels internals
  model_cooks = model_fit.get_influence().cooks_distance[0]
  tmp = pd.concat([model_fitted_y, model_residuals], axis=1, keys=['fitted', 'residuals'])


  plot_lm_1 = plt.figure()
  plot_lm_1.axes[0] = sns.residplot(data=tmp, x='fitted', y='residuals',
                            lowess=True,
                            scatter_kws={'alpha': 0.5},
                            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

  plot_lm_1.axes[0].set_title('Residuals vs Fitted')
  plot_lm_1.axes[0].set_xlabel('Fitted values')
  plot_lm_1.axes[0].set_ylabel('Residuals');

  # annotations
  abs_resid = model_abs_resid.sort_values(ascending=False)

  QQ = ProbPlot(model_norm_residuals)
  plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
  plot_lm_2.axes[0].set_title('Normal Q-Q')
  plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
  plot_lm_2.axes[0].set_ylabel('Standardized Residuals');
  # annotations
  abs_norm_resid = np.flip(np.argsort(np.abs(model_norm_residuals)), 0)

  plot_lm_3 = plt.figure()
  plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5);
  tmp2 = pd.DataFrame(model_norm_residuals_abs_sqrt, columns = ['model_norm_residuals_abs_sqrt'])
  tmp3 = pd.concat([model_fitted_y, tmp2], axis=1, keys=['fitted', 'residuals_normed'])
  sns.regplot(data=tmp3, x='fitted', y='residuals_normed',
                scatter=False,
                ci=False,
                lowess=True,
                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
  plot_lm_3.axes[0].set_title('Scale-Location')
  plot_lm_3.axes[0].set_xlabel('Fitted values')
  plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$');

  # annotations
  abs_sq_norm_resid = np.flip(np.argsort(model_norm_residuals_abs_sqrt), 0)


  plot_lm_4 = plt.figure();
  plt.scatter(model_leverage, model_norm_residuals, alpha=0.5);
  tmp4 = pd.DataFrame(model_leverage, columns = ['model_leverage'])
  tmp5 = pd.DataFrame(model_norm_residuals, columns = ['model_norm_residuals'])
  tmp6 = pd.concat([tmp4, tmp5], axis=1, keys=['leverage', 'residuals_normed'])
  sns.regplot(data=tmp6, x='leverage', y='residuals_normed',
             scatter=False,
             ci=False,
             lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
  plot_lm_4.axes[0].set_xlim(0, max(model_leverage)+0.01)
  plot_lm_4.axes[0].set_ylim(-3, 5)
  plot_lm_4.axes[0].set_title('Residuals vs Leverage')
  plot_lm_4.axes[0].set_xlabel('Leverage')
  plot_lm_4.axes[0].set_ylabel('Standardized Residuals');

  # annotations
  leverage_top_3 = np.flip(np.argsort(model_cooks), 0)[:3]
  for i in leverage_top_3:
      plot_lm_4.axes[0].annotate(i,
                                 xy=(model_leverage[i],
                                     model_norm_residuals[i]));

  p = len(model_fit.params) # number of model parameters
  graph(lambda x: np.sqrt((0.5 * p * (1 - x)) / x),
        np.linspace(0.001, max(model_leverage), 50),
        'Cook\'s distance') # 0.5 line
  graph(lambda x: np.sqrt((1 * p * (1 - x)) / x),
        np.linspace(0.001, max(model_leverage), 50)) # 1 line
  plot_lm_4.legend(loc='upper right');

In [0]:
X = df_clean["green pigments: index"]
y = df_clean["green pigments: direct concentration measurement (ug/g)"]

# Fit model to get leverage values and row indices
import statsmodels.api as sm
model_fit = sm.OLS(y, sm.add_constant(X)).fit()
influence = model_fit.get_influence()
leverage = influence.hat_matrix_diag
n = len(df_clean)
p = 2  # intercept + 1 predictor
leverage_thresh = 2 * p / n
high_leverage_idx = np.where(leverage > leverage_thresh)[0]
row_indices = df_clean.index.to_numpy()

def diagnostic_plots_with_labels(X, y, model_fit, high_lev_idx, row_indices):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from statsmodels.graphics.gofplots import ProbPlot
    import numpy as np
    model_fitted_y = model_fit.fittedvalues
    model_residuals = model_fit.resid
    model_norm_residuals = model_fit.get_influence().resid_studentized_internal
    model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
    model_leverage = model_fit.get_influence().hat_matrix_diag

    # 1. Residuals vs Fitted
    plt.figure()
    sns.residplot(x=model_fitted_y, y=model_residuals, lowess=True,
                  scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    plt.title('Residuals vs Fitted')
    plt.xlabel('Fitted values')
    plt.ylabel('Residuals')
    for i in high_lev_idx:
        plt.annotate(str(row_indices[i]), (model_fitted_y.iloc[i], model_residuals.iloc[i]), fontsize=8, color='red')

    # 2. Normal Q-Q
    QQ = ProbPlot(model_norm_residuals)
    fig = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
    plt.title('Normal Q-Q')
    plt.xlabel('Theoretical Quantiles')
    plt.ylabel('Standardized Residuals')
    for i in high_lev_idx:
        plt.annotate(str(row_indices[i]), (np.sort(QQ.theoretical_quantiles)[i], np.sort(model_norm_residuals)[i]), fontsize=8, color='red')

    # 3. Scale-Location
    plt.figure()
    plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5)
    sns.regplot(x=model_fitted_y, y=model_norm_residuals_abs_sqrt, scatter=False, ci=False, lowess=True,
                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    plt.title('Scale-Location')
    plt.xlabel('Fitted values')
    plt.ylabel('$\sqrt{|Standardized Residuals|}$')
    for i in high_lev_idx:
        plt.annotate(str(row_indices[i]), (model_fitted_y.iloc[i], model_norm_residuals_abs_sqrt[i]), fontsize=8, color='red')

    # 4. Residuals vs Leverage
    plt.figure()
    plt.scatter(model_leverage, model_norm_residuals, alpha=0.5)
    sns.regplot(x=model_leverage, y=model_norm_residuals, scatter=False, ci=False, lowess=True,
                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    plt.xlim(0, max(model_leverage)+0.01)
    plt.ylim(-3, 5)
    plt.title('Residuals vs Leverage')
    plt.xlabel('Leverage')
    plt.ylabel('Standardized Residuals')
    for i in high_lev_idx:
        plt.annotate(str(row_indices[i]), (model_leverage[i], model_norm_residuals[i]), fontsize=8, color='red')
    # Cook's distance lines
    def graph(formula, x_range, label=None):
        x = x_range
        y = formula(x)
        plt.plot(x, y, label=label, lw=1, ls='--', color='red')
    p = len(model_fit.params)
    graph(lambda x: np.sqrt((0.5 * p * (1 - x)) / x),
          np.linspace(0.001, max(model_leverage), 50),
          'Cook\'s distance')
    graph(lambda x: np.sqrt((1 * p * (1 - x)) / x),
          np.linspace(0.001, max(model_leverage), 50))
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

diagnostic_plots_with_labels(X, y, model_fit, high_leverage_idx, row_indices)

In [0]:
import scipy.stats as stats
import matplotlib.pyplot as plt

# Log-transform y
y_log = np.log(df_clean["green pigments: direct concentration measurement (ug/g)"])

# QQ plot for log-transformed y
with plt.style.context("seaborn"):
    fig, ax = plt.subplots(figsize=(8, 6))
    stats.probplot(y_log, dist="norm", plot=ax)
    ax.set_title("QQ Plot (log-transformed y)", fontsize=14)
    plt.tight_layout()
    plt.show()

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Remove missing values and log-transform X, filter out inf/-inf after log
df_model = df_clean.dropna(subset=["green pigments: direct concentration measurement (ug/g)", "organic carbon content in %"]).copy()
df_model["log_x"] = np.log(df_model["green pigments: direct concentration measurement (ug/g)"])
df_model = df_model.replace([np.inf, -np.inf], np.nan).dropna(subset=["log_x", "organic carbon content in %"])

# 1. Train/test split
X = df_model[["log_x"]]
y = df_model["organic carbon content in %"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Fit model
model = LinearRegression()
model.fit(X_train, y_train)

# 3. Evaluate metrics
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

metrics = {
    "RMSE_train": mean_squared_error(y_train, y_train_pred, squared=False),
    "RMSE_test": mean_squared_error(y_test, y_test_pred, squared=False),
    "MAE_train": mean_absolute_error(y_train, y_train_pred),
    "MAE_test": mean_absolute_error(y_test, y_test_pred),
    "R2_train": r2_score(y_train, y_train_pred),
    "R2_test": r2_score(y_test, y_test_pred)
}
print(metrics)

# 4. Plot y vs predicted y for train and test
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].scatter(y_train, y_train_pred, alpha=0.7)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
axes[0].set_title("Train: Actual vs Predicted")
axes[0].set_xlabel("Actual y")
axes[0].set_ylabel("Predicted y")

axes[1].scatter(y_test, y_test_pred, alpha=0.7)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[1].set_title("Test: Actual vs Predicted")
axes[1].set_xlabel("Actual y")
axes[1].set_ylabel("Predicted y")

plt.tight_layout()
plt.show()


**Model Evaluation (English):**

In my opinion, the model does not work properly. It always predicts a CO2 concentration of around 25%, regardless of the input. This is not realistic. In reality, CO2 concentrations above 50% were often observed. I am not surprised by this, because in reality, CO2 concentration is likely influenced by many more factors than just the pigment content. Therefore, in my view, the model is too simplistic.